In [4]:
import reaktoro as rkt 

In [5]:
# create database object
db = rkt.Database('supcrt98.xml')

In [32]:
# Define the chemical system
editor = rkt.ChemicalEditor(db)

# list of aqueous species to add to the system
aqueous_species = ['H2O(l)', 'H+', 'OH-', 'O2(aq)', 'H2(aq)', 'Cl-', 
                   'HCl(aq)', 'ClO4-', 'Na+', 'NaOH(aq)', 'NaCl(aq)']

# set extended Debye-Huckel model parameters according to b-gamma model
DH_model = rkt.DebyeHuckelParams()
for species in aqueous_species:
    if db.aqueousSpecies(species).charge() != 0 : # only change model parameters for charged species
        DH_model.aion(species, 3.72) # constant ion size parameter
        DH_model.bion(species, 0.0201) # constant b-gamma parameter calculated w/ HCh for NaCl background electrolyte @ 250 °C, P_sat

# define aqueous phase()
aqueous_phase = editor.addAqueousPhase(aqueous_species)
aqueous_phase.setChemicalModelDebyeHuckel()

# Construct the chemical system
system = rkt.ChemicalSystem(editor)

In [33]:
problem = rkt.EquilibriumProblem(system)

# Set T,P conditions
problem.setTemperature(250+273.15, "kelvin")
problem.setPressure(rkt.waterSaturatedPressureWagnerPruss(problem.temperature()).val, "Pa")

# Add components to system
problem.add("H2O(l)", 1, "kg")
problem.add("NaCl(aq)", 0.1, "mol")

# Solve the inverse equilibrium problem
state = rkt.equilibrate(problem)

print(state)

Temperature [K]          Temperature [C]          Pressure [Pa]            Pressure [bar]           
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
523.15                   250                      3.9762e+06               39.762                   
Element                  Amount [mol]             Aqueous [mol]            Dual Potential [kJ/mol]  
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Cl                       0.1                      0.1                      -119.382                 
H                        111.017                  111.017                  -82.7931                 
Na                       0.1               